In [1]:
import json
import os

import pandas as pd

,Tahun,Bulan,Provinsi,Bawang Merah,Bawang Putih,Beras,Cabai Merah,Cabai Rawit,Daging Ayam,Daging Sapi,Gula Pasir,Minyak Goreng,Telur Ayam,Bulan_y,index
0,2016,2,ACEH,34725.0,22425.0,10137.5,56387.5,41562.5,28800.0,126037.5,16387.5,14162.5,22400.0,1,0.868508
1,2016,2,BALI,37662.5,31687.5,10625.0,32137.5,30450.0,32537.5,101750.0,15600.0,13600.0,18462.5,1,-0.051571
2,2016,2,BANTEN,42212.5,36400.0,10862.5,49087.5,40375.0,32662.5,113737.5,15375.0,12312.5,21137.5,1,0.410912
3,2016,2,BENGKULU,39625.0,36875.0,10375.0,51700.0,45375.0,32250.0,126250.0,15975.0,13012.5,20462.5,1,-0.089437
4,2016,2,DI YOGYAKARTA,38937.5,35037.5,10637.5,39975.0,33875.0,30437.5,114400.0,15025.0,12325.0,18800.0,1,0.297294


In [15]:
with open(os.path.join('input', 'indonesia-prov.json')) as f:
    province_data = json.load(f)['features']

province_ids = {}
for province in province_data:
    properties = province['properties']
    
    province_key = properties['Propinsi'].lower()
        
    province_ids[province_key] = properties['ID']

province_ids['semua provinsi'] = -1
province_ids

{'kalimantan timur': 64,
 'kalimantan utara': 65,
 'jawa barat': 32,
 'jambi': 15,
 'gorontalo': 75,
 'dki jakarta': 31,
 'di yogyakarta': 34,
 'bengkulu': 17,
 'banten': 36,
 'bali': 51,
 'aceh': 11,
 'jawa tengah': 33,
 'jawa timur': 35,
 'kalimantan barat': 61,
 'kalimantan selatan': 63,
 'kalimantan tengah': 62,
 'kepulauan bangka belitung': 19,
 'kepulauan riau': 21,
 'lampung': 18,
 'maluku': 81,
 'maluku utara': 82,
 'nusa tenggara barat': 52,
 'nusa tenggara timur': 53,
 'papua': 91,
 'papua barat': 94,
 'riau': 14,
 'sulawesi barat': 76,
 'sulawesi selatan': 73,
 'sulawesi tengah': 72,
 'sulawesi tenggara': 74,
 'sulawesi utara': 71,
 'sumatera barat': 13,
 'sumatera selatan': 16,
 'sumatera utara': 12,
 'semua provinsi': -1}

In [16]:
merged_food_prices['Provinsi'] = merged_food_prices['Provinsi'].apply(lambda x : x.lower()).replace(province_ids)

In [17]:
merged_food_prices.to_csv(
    os.path.join('output', 'food_data.csv'),
    index=False
)

In [18]:
merged_food_prices = pd.read_csv(
    os.path.join('output', 'food_data.csv')
)
merged_food_prices.head()

,Tahun,Bulan,Provinsi,Bawang Merah,Bawang Putih,Beras,Cabai Merah,Cabai Rawit,Daging Ayam,Daging Sapi,Gula Pasir,Minyak Goreng,Telur Ayam,Bulan_y,index
0,2016,2,11,34725.0,22425.0,10137.5,56387.5,41562.5,28800.0,126037.5,16387.5,14162.5,22400.0,1,0.868508
1,2016,2,51,37662.5,31687.5,10625.0,32137.5,30450.0,32537.5,101750.0,15600.0,13600.0,18462.5,1,-0.051571
2,2016,2,36,42212.5,36400.0,10862.5,49087.5,40375.0,32662.5,113737.5,15375.0,12312.5,21137.5,1,0.410912
3,2016,2,17,39625.0,36875.0,10375.0,51700.0,45375.0,32250.0,126250.0,15975.0,13012.5,20462.5,1,-0.089437
4,2016,2,34,38937.5,35037.5,10637.5,39975.0,33875.0,30437.5,114400.0,15025.0,12325.0,18800.0,1,0.297294


In [19]:
len(merged_food_prices)

1540

In [20]:
month_dict = {
    '1': 'JAN',
    '2': 'FEB',
    '3': 'MAR',
    '4': 'APR',
    '5': 'MEI',
    '6': 'JUN',
    '7': 'JUL',
    '8': 'AUG',
    '9': 'SEP',
    '10': 'OKT',
    '11': 'NOV',
    '12': 'DES'
}

In [21]:
json_data = {}
merged_food_prices['Bulan'] = merged_food_prices['Bulan'].astype(str).replace(month_dict)

merged_food_prices['label'] = (
    merged_food_prices['Bulan'].apply(str) + '-'
    + merged_food_prices['Tahun'].apply(str)
)

for period in pd.unique(merged_food_prices['label']):
    period_data = merged_food_prices.loc[
        merged_food_prices['label'] == period
    ].reset_index(drop=True)
    
    period_dict = {}
    
    for location in pd.unique(merged_food_prices['Provinsi']):
        location_data = period_data.loc[
            period_data['Provinsi'] == location
        ]
        location_dict = {}
        
        for col in location_data.columns:
            if location_data[col].dtype != 'float64':
                continue
            location_dict[col] = float(location_data[col].values[0])
        
        period_dict[str(location)] = location_dict
    
    json_data[period] = period_dict
    
with open(os.path.join('output', 'data.json'), 'w') as f:
    json.dump(json_data, f)